In [56]:
# Importing libraries
import pandas as pd
import numpy as np
import pickle
pd.options.mode.copy_on_write = True 

In [57]:
# Helper function
def to_datetime(df):
    df['match_dt'] = pd.to_datetime(df['match_dt'], format='%Y-%m-%d')
    return df

def rm_blankspace(df):
    df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
    return df

def unwrap_rosters(df):
    df['team1_roster_ids'] = df['team1_roster_ids'].apply(lambda x: x.split(':'))
    df['team2_roster_ids'] = df['team2_roster_ids'].apply(lambda x: x.split(':'))
    return df

def data_preprocessing(df):
    df = to_datetime(df)
    df = rm_blankspace(df)
    return df

In [58]:
# Importing datset
batsman_data = pd.read_csv('data//batsman_level_data.csv')
bowler_data = pd.read_csv('data//bowler_level_data.csv')
match_data = pd.read_csv('data//match_level_data.csv')
test_data = pd.read_csv('data//round_2_sub_data.csv')

In [59]:
match_data = data_preprocessing(match_data)
match_data = unwrap_rosters(match_data)
test_data = data_preprocessing(test_data)
test_data = unwrap_rosters(test_data)

batsman_data = data_preprocessing(batsman_data)
bowler_data = data_preprocessing(bowler_data)

In [60]:
# Reordering columns
match_data['toss_winner_id'] = np.where(match_data['toss_winner'] == match_data['team1'], match_data['team1_id'], match_data['team2_id'])
match_data = match_data[['match_id', 'match_dt', 'team1', 'team1_id', 'team2', 'team2_id',
        'toss_winner_id', 'toss_decision', 'ground_id', 'lighting', 'series_name',
        'winner_id', 'by', 'win_amount', 'player_of_the_match_id',
        'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls',
        'team1_roster_ids', 'team2_roster_ids']]
match_data['team_roster_ids'] = match_data['team1_roster_ids'] + match_data['team2_roster_ids']

test_data = test_data[['match_id', 'match_dt', 'team1', 'team1_id', 'team2', 'team2_id',
       'toss_winner', 'toss_decision', 'ground_id', 'lighting', 'series_name',
       'team1_roster_ids', 'team2_roster_ids', 'team_count_50runs_last15',
       'team_winp_last5', 'team1only_avg_runs_last15', 'team1_winp_team2_last15','ground_avg_runs_last15']]
test_data['team_roster_ids'] = test_data['team1_roster_ids'] + test_data['team2_roster_ids']

test_data.rename(columns={'team_count_50runs_last15': 'team1Num50Last15', 'team_winp_last5': 'team1WinpLast5', 'team1only_avg_runs_last15':'team1OnlyAvgRunsLast15',
                           'team1_winp_team2_last15':'team1WinpLast15F2F','ground_avg_runs_last15':'groundAvgRunsLast15'}, inplace=True)

In [61]:
# Cheatsheet features
def ngiveLastNgamesPlayer(player_id, date, bat_or_bowl, n):

    if bat_or_bowl == 'bat':
        df_topick = batsman_data
        id_col = 'batsman_id'
    else:
        df_topick = bowler_data
        id_col = 'bowler_id'
        
    return df_topick[(df_topick['match_dt']<date)&(df_topick[id_col]==float(player_id))]\
                .sort_values(by='match_dt', ascending=False).head(n)

def giveLastNgamesPlayer(player_id, date, bat_or_bowl):

    if bat_or_bowl == 'bat':
        df_topick = batsman_data
        id_col = 'batsman_id'
    else:
        df_topick = bowler_data
        id_col = 'bowler_id'
        
    return df_topick[(df_topick['match_dt']<date)&(df_topick[id_col]==float(player_id))]\
                .sort_values(by='match_dt', ascending=False)

def avgRunsGround(ground_id, date, n):
    df_rel = match_data[(match_data['match_dt']<date)&(match_data['ground_id']==ground_id)].sort_values(by='match_dt', ascending=False).head(n)
    df_rel['avg_runs_inn'] = (df_rel['inning1_runs']+df_rel['inning2_runs'])/2
    return df_rel['avg_runs_inn'].mean()


In [62]:
# Ratio invariant features
def avgWicketsGround(ground_id, date, n):
    df_rel = match_data[(match_data['match_dt']<date)&(match_data['ground_id']==ground_id)].sort_values(by='match_dt', ascending=False).head(n)
    df_rel['avg_wickets_inn'] = (df_rel['inning1_wickets']+df_rel['inning2_wickets'])/2
    return df_rel['avg_wickets_inn'].mean()

def lightAvgRunsLast15(light, date, n):
    df_rel = match_data[(match_data['match_dt']<date)&(match_data['lighting']==light)].sort_values(by='match_dt', ascending=False).head(n)
    df_rel['avg_runs_inn'] = (df_rel['inning1_runs']+df_rel['inning2_runs'])/2
    return df_rel['avg_runs_inn'].mean()

def lightAvgWicketsLast15(light, date, n):
    df_rel = match_data[(match_data['match_dt']<date)&(match_data['lighting']==light)].sort_values(by='match_dt', ascending=False).head(n)
    df_rel['avg_wickets_inn'] = (df_rel['inning1_wickets']+df_rel['inning2_wickets'])/2
    return df_rel['avg_wickets_inn'].mean()

def seriesAvgRunsLast15(series, date, n):
    df_rel = match_data[(match_data['match_dt']<date)&(match_data['series_name']==series)].sort_values(by='match_dt', ascending=False).head(n)
    df_rel['avg_runs_inn'] = (df_rel['inning1_runs']+df_rel['inning2_runs'])/2
    return df_rel['avg_runs_inn'].mean()

def seriesAvgWicketsLast15(series, date, n):
    df_rel = match_data[(match_data['match_dt']<date)&(match_data['series_name']==series)].sort_values(by='match_dt', ascending=False).head(n)
    df_rel['avg_wickets_inn'] = (df_rel['inning1_wickets']+df_rel['inning2_wickets'])/2
    return df_rel['avg_wickets_inn'].mean()

def inn1AvgRunsLast15(date, n):
    df_rel = match_data[(match_data['match_dt']<date)].sort_values(by='match_dt', ascending=False).head(n)
    return df_rel['inning1_runs'].mean()

def inn2AvgRunsLast15(date, n):
    df_rel = match_data[(match_data['match_dt']<date)].sort_values(by='match_dt', ascending=False).head(n)
    return df_rel['inning2_runs'].mean()

def inn1AvgWicketsLast15(date, n):
    df_rel = match_data[(match_data['match_dt']<date)].sort_values(by='match_dt', ascending=False).head(n)
    return df_rel['inning1_wickets'].mean()

def inn2AvgWicketsLast15(date, n):
    df_rel = match_data[(match_data['match_dt']<date)].sort_values(by='match_dt', ascending=False).head(n)
    return df_rel['inning2_wickets'].mean()

In [63]:
# Match level features

def team1tossWinnerWins(team1_id, team2_id, date):

    match_data['DtossWinnerWins'] = np.where(match_data['toss_winner_id'] == match_data['winner_id'], 1, 0)
    df_rel1 = match_data[(match_data['match_dt']<date)&\
                        (((match_data['team1_id']==team1_id)|(match_data['team2_id']==team1_id)))]\
                        .sort_values(by='match_dt', ascending=False)
    
    df_rel2 = match_data[(match_data['match_dt']<date)&\
                        (((match_data['team1_id']==team2_id)|(match_data['team2_id']==team2_id)))]\
                        .sort_values(by='match_dt', ascending=False)
    
    win_count1 = df_rel1[df_rel1['DtossWinnerWins'] == 1]
    win_count1 = win_count1[win_count1['winner_id'] == team1_id]

    win_count2 = df_rel2[df_rel2['DtossWinnerWins'] == 1]
    win_count2 = win_count2[win_count2['winner_id'] == team2_id]

    if len(df_rel1)==0 or len(df_rel2)==0 or len(win_count1) == 0 or len(win_count2) ==0:
        return 1
    else:
        return round((len(win_count1)/len(df_rel1))/(len(win_count2)/len(df_rel2)),2)
  
def team1BatsFirstWins(team1_id, team2_id, date):
    
    match_data['toss_decision_bats'] = np.where(match_data['toss_decision'] == 'bat', 1, 0)

    match_data['DteamBatsFirstWins'] = np.where(((match_data['DtossWinnerWins'] == 1) & (match_data['toss_decision_bats'] == 1))
                                          | ((match_data['DtossWinnerWins'] == 0) & (match_data['toss_decision_bats'] == 0)), 1, 0)

    df_rel1 = match_data[(match_data['match_dt']<date)&\
                        (((match_data['team1_id']==team1_id)|(match_data['team2_id']==team1_id)))]\
                        .sort_values(by='match_dt', ascending=False)
    
    df_rel2 = match_data[(match_data['match_dt']<date)&\
                        (((match_data['team1_id']==team2_id)|(match_data['team2_id']==team2_id)))]\
                        .sort_values(by='match_dt', ascending=False)
        
    win_count1 = df_rel1[df_rel1['DteamBatsFirstWins'] == 1]
    win_count1 = win_count1[win_count1['winner_id'] == team1_id]

    win_count2 = df_rel2[df_rel2['DteamBatsFirstWins'] == 1]
    win_count2 = win_count2[win_count2['winner_id'] == team2_id]

    if len(df_rel1)==0 or len(df_rel2)==0 or len(win_count1) == 0 or len(win_count2) ==0:
        return 1
    else:
        return round((len(win_count1)/len(df_rel1))/(len(win_count2)/len(df_rel2)),2)

def team1BatsSecondWins(team1_id, team2_id, date):

    match_data['DteamBatsSecondWins'] = np.where(((match_data['DtossWinnerWins'] == 1) & (match_data['toss_decision_bats'] == 0))
                                          | ((match_data['DtossWinnerWins'] == 0) & (match_data['toss_decision_bats'] == 1)), 1, 0)

    df_rel1 = match_data[(match_data['match_dt']<date)&\
                        (((match_data['team1_id']==team1_id)|(match_data['team2_id']==team1_id)))]\
                        .sort_values(by='match_dt', ascending=False)
    
    df_rel2 = match_data[(match_data['match_dt']<date)&\
                        (((match_data['team1_id']==team2_id)|(match_data['team2_id']==team2_id)))]\
                        .sort_values(by='match_dt', ascending=False)
        
    win_count1 = df_rel1[df_rel1['DteamBatsSecondWins'] == 1]
    win_count1 = win_count1[win_count1['winner_id'] == team1_id]

    win_count2 = df_rel2[df_rel2['DteamBatsSecondWins'] == 1]
    win_count2 = win_count2[win_count2['winner_id'] == team2_id]

    if len(df_rel1)==0 or len(df_rel2)==0 or len(win_count1) == 0 or len(win_count2) ==0:
        return 1
    else:
        return round((len(win_count1)/len(df_rel1))/(len(win_count2)/len(df_rel2)),2)

def team1WinpLight(team1_id, team2_id, date, light):

    df_rel1 = match_data[(match_data['match_dt']<date)&\
                        (match_data['lighting'] == light)&\
                      (((match_data['team1_id']==team1_id)|(match_data['team2_id']==team1_id)))]\
                        .sort_values(by='match_dt', ascending=False)

    df_rel2 = match_data[(match_data['match_dt']<date)&\
                        (match_data['lighting'] == light)&\
                      (((match_data['team1_id']==team2_id)|(match_data['team2_id']==team2_id)))]\
                        .sort_values(by='match_dt', ascending=False)
 
    win_count1 = df_rel1[df_rel1['winner_id']==team1_id]
    win_count2 = df_rel2[df_rel2['winner_id']==team2_id]

    if len(df_rel1) == 0 or len(df_rel2) == 0 or len(win_count1) == 0 or len(win_count2) == 0:
        return 1
    else:
        return round((len(win_count1)/len(df_rel1))/(len(win_count2)/len(df_rel2)),2)

def team1WinpSeries(team1_id, team2_id, date, series):

    df_rel1 = match_data[(match_data['match_dt']<date)&\
                        (match_data['series_name'] == series)&\
                      (((match_data['team1_id']==team1_id)|(match_data['team2_id']==team1_id)))]\
                        .sort_values(by='match_dt', ascending=False)

    df_rel2 = match_data[(match_data['match_dt']<date)&\
                        (match_data['series_name'] == series)&\
                      (((match_data['team1_id']==team2_id)|(match_data['team2_id']==team2_id)))]\
                        .sort_values(by='match_dt', ascending=False)
    
    win_count1 = df_rel1[df_rel1['winner_id']==team1_id]
    win_count2 = df_rel2[df_rel2['winner_id']==team2_id]

    if len(df_rel1) == 0 or len(df_rel2) == 0 or len(win_count1) == 0 or len(win_count2) == 0:
        return 1
    else:
        return round((len(win_count1)/len(df_rel1))/(len(win_count2)/len(df_rel2)),2)

def team1AvgRunsMargin(team1_id, team2_id, date, n):

    df_rel1 = match_data[(match_data['match_dt']<date)&\
                      ((match_data['team1_id']==team1_id)|(match_data['team2_id']==team1_id))&\
                       (match_data['winner_id']==team1_id)&\
                        (match_data['by']=='runs')]\
                        .sort_values(by='match_dt', ascending=False).head(n)
    
    df_rel2 = match_data[(match_data['match_dt']<date)&\
                      ((match_data['team1_id']==team2_id)|(match_data['team2_id']==team2_id))&\
                       (match_data['winner_id']==team2_id)&\
                        (match_data['by']=='runs')]\
                        .sort_values(by='match_dt', ascending=False).head(n)

    if len(df_rel1)==0 or len(df_rel2)==0:
        return 1
    else:
        return np.mean(df_rel1['win_amount'])/np.mean(df_rel2['win_amount'])
    
def team1AvgWicketsMargin(team1_id, team2_id, date, n):

    df_rel1 = match_data[(match_data['match_dt']<date)&\
                      ((match_data['team1_id']==team1_id)|(match_data['team2_id']==team1_id))&\
                       (match_data['winner_id']==team1_id)&\
                        (match_data['by']=='wickets')]\
                        .sort_values(by='match_dt', ascending=False).head(n)
    
    df_rel2 = match_data[(match_data['match_dt']<date)&\
                      ((match_data['team1_id']==team2_id)|(match_data['team2_id']==team2_id))&\
                       (match_data['winner_id']==team2_id)&\
                        (match_data['by']=='wickets')]\
                        .sort_values(by='match_dt', ascending=False).head(n)

    if len(df_rel1)==0 or len(df_rel2)==0:
        return 1
    else:
        return np.mean(df_rel1['win_amount'])/np.mean(df_rel2['win_amount'])

def team1AvgWicketsLost(team1_id, team2_id, date, n):

    df_rel1 = match_data[(match_data['match_dt']<date)&\
                      ((match_data['team1_id']==team1_id)|(match_data['team2_id']==team1_id))]\
                        .sort_values(by='match_dt', ascending=False).head(n)
    df_rel1['team1'] = team1_id

    df_rel2 = match_data[(match_data['match_dt']<date)&\
                      ((match_data['team1_id']==team2_id)|(match_data['team2_id']==team2_id))]\
                        .sort_values(by='match_dt', ascending=False).head(n)
    df_rel2['team2'] = team2_id

    df_rel1['bat_inning'] = np.where( ((df_rel1['team1']==df_rel1['toss_winner_id'])&(df_rel1['toss_decision']=='bat'))|\
                                            ((df_rel1['team1']!=df_rel1['toss_winner_id'])&(df_rel1['toss_decision']=='field')) , 1, 2)

    df_rel2['bat_inning'] = np.where( ((df_rel2['team2']==df_rel2['toss_winner_id'])&(df_rel2['toss_decision']=='bat'))|\
                                            ((df_rel2['team2']!=df_rel2['toss_winner_id'])&(df_rel2['toss_decision']=='field')) , 1, 2)
    
    df_rel1 = df_rel1[df_rel1['bat_inning']==1][['inning1_wickets', 'inning1_balls']].rename(columns={'inning1_wickets':'wickets', 'inning1_balls':'balls'})
    df_rel2 = df_rel2[df_rel2['bat_inning']==1][['inning1_wickets', 'inning1_balls']].rename(columns={'inning1_wickets':'wickets', 'inning1_balls':'balls'})
    
    df_rel1['ball_per_wick'] = df_rel1['balls']/(df_rel1['wickets']+1)
    df_rel2['ball_per_wick'] = df_rel2['balls']/(df_rel2['wickets']+1)

    if len(df_rel1)==0 or len(df_rel2)==0:
        return 1
    else:
        return df_rel1['ball_per_wick'].mean()/df_rel2['ball_per_wick'].mean()

def team1AvgRR(team1_id, team2_id, date, n):

    df_rel1 = match_data[(match_data['match_dt']<date)&\
                      ((match_data['team1_id']==team1_id)|(match_data['team2_id']==team1_id))]\
                        .sort_values(by='match_dt', ascending=False).head(n)
    df_rel1['team1'] = team1_id

    df_rel2 = match_data[(match_data['match_dt']<date)&\
                      ((match_data['team1_id']==team2_id)|(match_data['team2_id']==team2_id))]\
                        .sort_values(by='match_dt', ascending=False).head(n)
    df_rel2['team2'] = team2_id

    df_rel1['bat_inning'] = np.where( ((df_rel1['team1']==df_rel1['toss_winner_id'])&(df_rel1['toss_decision']=='bat'))|\
                                            ((df_rel1['team1']!=df_rel1['toss_winner_id'])&(df_rel1['toss_decision']=='field')) , 1, 2)

    df_rel2['bat_inning'] = np.where( ((df_rel2['team2']==df_rel2['toss_winner_id'])&(df_rel2['toss_decision']=='bat'))|\
                                            ((df_rel2['team2']!=df_rel2['toss_winner_id'])&(df_rel2['toss_decision']=='field')) , 1, 2)
    
    df_rel1 = pd.concat([ df_rel1[df_rel1['bat_inning']==1][['inning1_runs', 'inning1_balls']].rename(columns={'inning1_runs':'runs', 'inning1_balls':'balls'}), \
                         df_rel1[df_rel1['bat_inning']==2][['inning2_runs', 'inning2_balls']].rename(columns={'inning2_runs':'runs', 'inning1_balls':'balls'}) ] )
    
    df_rel1['RR'] = df_rel1['runs']/df_rel1['balls']
    
    df_rel2 = pd.concat([ df_rel2[df_rel2['bat_inning']==1][['inning1_runs', 'inning1_balls']].rename(columns={'inning1_runs':'runs', 'inning1_balls':'balls'}), \
                         df_rel2[df_rel2['bat_inning']==2][['inning2_runs', 'inning2_balls']].rename(columns={'inning2_runs':'runs', 'inning1_balls':'balls'}) ] )
    
    df_rel2['RR'] = df_rel2['runs']/df_rel2['balls']

    if len(df_rel1)==0 or len(df_rel2)==0:
        return 1
    else:
        return df_rel1['RR'].mean()/df_rel2['RR'].mean()

In [64]:
# Player level features

def team1PlayerOfMatchLast15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)
        df_rel1 = df_rel1.merge(match_data[match_data['match_dt']<date], on='match_id', how='left')[['player_of_the_match_id']]
        df_rel1['isPlayerOfMatch'] = np.where(df_rel1['player_of_the_match_id']==float(player), 1, 0)
        res_list1.append(np.sum(df_rel1['isPlayerOfMatch']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)
        df_rel2 = df_rel2.merge(match_data[match_data['match_dt']<date], on='match_id', how='left')[['player_of_the_match_id']]
        df_rel2['isPlayerOfMatch'] = np.where(df_rel2['player_of_the_match_id']==float(player), 1, 0)
        res_list2.append(np.sum(df_rel2['isPlayerOfMatch']))
    
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)

def team1TotalRunsPOMLast15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)
        df_rel1 = df_rel1.merge(match_data[match_data['match_dt']<date], on='match_id', how='left')[['player_of_the_match_id', 'runs']]
        df_rel1['isPlayerOfMatch'] = np.where(df_rel1['player_of_the_match_id']==float(player), 1, 0)
        df_rel1 = df_rel1[df_rel1['isPlayerOfMatch']==1]
        res_list1.append(np.sum(df_rel1['runs']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)
        df_rel2 = df_rel2.merge(match_data[match_data['match_dt']<date], on='match_id', how='left')[['player_of_the_match_id', 'runs']]
        df_rel2['isPlayerOfMatch'] = np.where(df_rel2['player_of_the_match_id']==float(player), 1, 0)
        df_rel2 = df_rel2[df_rel2['isPlayerOfMatch']==1]
        res_list2.append(np.sum(df_rel2['runs']))
    
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)

def team1TotalWicketsPOMLast15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl', n=n)
        df_rel1 = df_rel1.merge(match_data[match_data['match_dt']<date], on='match_id', how='left')[['player_of_the_match_id', 'wicket_count']]
        df_rel1['isPlayerOfMatch'] = np.where(df_rel1['player_of_the_match_id']==float(player), 1, 0)
        df_rel1 = df_rel1[df_rel1['isPlayerOfMatch']==1]
        res_list1.append(np.sum(df_rel1['wicket_count']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl', n=n)
        df_rel2 = df_rel2.merge(match_data[match_data['match_dt']<date], on='match_id', how='left')[['player_of_the_match_id', 'wicket_count']]
        df_rel2['isPlayerOfMatch'] = np.where(df_rel2['player_of_the_match_id']==float(player), 1, 0)
        df_rel2 = df_rel2[df_rel2['isPlayerOfMatch']==1]
        res_list2.append(np.sum(df_rel2['wicket_count']))
    
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)

def team1SRover120Last15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)[['strike_rate']]
        df_rel1['sr120'] = np.where(df_rel1['strike_rate']>120, 1, 0)
        if len(df_rel1)>0:
            res_list1.append(np.mean(df_rel1['sr120']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)[['strike_rate']]
        df_rel2['sr120'] = np.where(df_rel2['strike_rate']>120, 1, 0)
        if len(df_rel2)>0:
            res_list2.append(np.mean(df_rel2['sr120']))
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.mean(res_list1)/np.mean(res_list2)
    
def team1CaptainRuns(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')
        df_rel1 = df_rel1[df_rel1['is_batsman_captain'] == 1][['runs']]
        if len(df_rel1)>0:
            res_list1.append(np.sum(df_rel1['runs']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')
        df_rel2 = df_rel2[df_rel2['is_batsman_captain'] == 1][['runs']]
        if len(df_rel2)>0:
            res_list2.append(np.sum(df_rel2['runs']))

    if len(res_list2) == 0 or len(res_list1) == 0 or np.sum(res_list2) == 0 or np.sum(res_list2) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)
    
def team1BattingAvgLast15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)[['runs']]
        if len(df_rel1)>0:
            df_rel1['batting_avg'] = df_rel1['runs']/len(df_rel1)
            res_list1.append(np.sum(df_rel1['batting_avg']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)[['runs']]
        if len(df_rel2)>0:
            df_rel2['batting_avg'] = df_rel2['runs']/len(df_rel2)
            res_list2.append(np.sum(df_rel2['batting_avg']))
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)

def team1BoundaryLast15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)[['Fours', 'Sixes']]
        if len(df_rel1)>0:
            df_rel1['boundaries'] = df_rel1['Fours'] + df_rel1['Sixes']
            res_list1.append(np.sum(df_rel1['boundaries']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)[['Fours', 'Sixes']]
        if len(df_rel2)>0:
            df_rel2['boundaries'] = df_rel2['Fours'] + df_rel2['Sixes']
            res_list2.append(np.sum(df_rel2['boundaries']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)
    
def team1BatBoundaryPercentLast15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)[['Fours', 'Sixes', 'runs']]
        if len(df_rel1)>0:
            df_rel1['boundaries'] = ((4*df_rel1['Fours'] + 6*df_rel1['Sixes'])+1)/(df_rel1['runs']+1)
            res_list1.append(np.sum(df_rel1['boundaries']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=n)[['Fours', 'Sixes', 'runs']]
        if len(df_rel2)>0:
            df_rel2['boundaries'] = ((4*df_rel2['Fours'] + 6*df_rel2['Sixes'])+1)/(df_rel2['runs']+1)
            res_list2.append(np.sum(df_rel2['boundaries']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.mean(res_list1)/np.mean(res_list2)
    
def team1EconBelow8Last15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl', n=n)[['economy']]
        df_rel1['econ8'] = np.where(df_rel1['economy']<8, 1, 0)
        if len(df_rel1)>0:
            res_list1.append(np.mean(df_rel1['econ8']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl', n=n)[['economy']]
        df_rel2['econ8'] = np.where(df_rel2['economy']<8, 1, 0)
        if len(df_rel2)>0:
            res_list2.append(np.mean(df_rel2['econ8']))
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.mean(res_list1)/np.mean(res_list2)

def team1DotPercentLast15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl', n=n)[['dots', 'balls_bowled']]
        if len(df_rel1)>0:
            df_rel1['dot_percent'] = df_rel1['dots']/df_rel1['balls_bowled']
            res_list1.append(np.mean(df_rel1['dot_percent']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl', n=n)[['dots', 'balls_bowled']]
        if len(df_rel2)>0:
            df_rel2['dot_percent'] = df_rel2['dots']/df_rel2['balls_bowled']
            res_list2.append(np.mean(df_rel2['dot_percent']))
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.mean(res_list1)/np.mean(res_list2)
    
def team1BowlBoundaryPercentLast15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl', n=n)[['Fours', 'Sixes', 'runs']]
        if len(df_rel1)>0:
            df_rel1['boundaries'] = ((4*df_rel1['Fours'] + 6*df_rel1['Sixes'])+1)/(df_rel1['runs']+1)
            res_list1.append(np.mean(df_rel1['boundaries']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl', n=n)[['Fours', 'Sixes', 'runs']]
        if len(df_rel2)>0:
            df_rel2['boundaries'] = ((4*df_rel2['Fours'] + 6*df_rel2['Sixes'])+1)/(df_rel2['runs']+1)
            res_list2.append(np.mean(df_rel2['boundaries']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0 or np.mean(res_list2) == 0 or np.mean(res_list1) == 0:
        return 1
    else:
        return np.mean(res_list1)/np.mean(res_list2)

def team1MaidenPercent(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['maiden', 'balls_bowled']]
        if len(df_rel1)>0:
            df_rel1['maiden_percent'] = df_rel1['maiden']*6/df_rel1['balls_bowled']
            res_list1.append(np.mean(df_rel1['maiden_percent']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['maiden', 'balls_bowled']]
        if len(df_rel2)>0:
            df_rel2['maiden_percent'] = df_rel2['maiden']*6/df_rel2['balls_bowled']
            res_list2.append(np.mean(df_rel2['maiden_percent']))
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.mean(res_list1)/np.mean(res_list2)
    
def team1ExtrasPercentLast15(player_list1, player_list2, date, n):

    res_list1 = []
    for player in player_list1:
        df_rel1 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl', n=n)[['wides', 'noballs', 'balls_bowled']]
        if len(df_rel1)>0:
            df_rel1['extras_percent'] = (df_rel1['wides'] + df_rel1['noballs'])/df_rel1['balls_bowled']
            res_list1.append(np.mean(df_rel1['extras_percent']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl', n=n)[['wides', 'noballs', 'balls_bowled']]
        if len(df_rel2)>0:
            df_rel2['extras_percent'] = (df_rel2['wides'] + df_rel2['noballs'])/df_rel2['balls_bowled']
            res_list2.append(np.mean(df_rel2['extras_percent']))
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.mean(res_list1)/np.mean(res_list2)


In [65]:
# Team score features

def scale_scores(x):
    if np.max(x)==0:
        return x
    else:
        return x/np.max(x)
    
def batted_played(player, date):
    x = match_data[match_data['match_dt']<date].sort_values(by='match_dt', ascending=False)[['match_id', 'team_roster_ids']]
    x['player_played'] = x['team_roster_ids'].apply(lambda x: 1 if player in x else 0)
    matches_played = x[x['player_played'] == 1]['match_id'].values

    x = batsman_data[batsman_data['match_dt']<date].sort_values(by='match_dt', ascending=False)[['match_id', 'batsman_id']]
    matches_batted = x[x['batsman_id']==float(player)]['match_id'].values

    for i in matches_batted:
        if i not in matches_played:
            matches_batted = np.delete(matches_batted, np.where(matches_batted==i))
    
    return np.sqrt(len(matches_batted)/(len(matches_played)+1))

def cent_fifties(player, date):
    df_rel = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')
    df_rel['fifty'] = np.where((df_rel['runs']>=50) & (df_rel['runs']<100), 1, 0)
    df_rel['century'] = np.where(df_rel['runs']>=100, 1, 0)
    return 20*np.sum(df_rel['century']) + 5*np.sum(df_rel['fifty'])

def batting_avg(player, date):
    df_rel = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')
    return df_rel['runs'].sum()/(len(df_rel)+1)

def batsmanScore(player_list, date):
    career_score_list = []
    recent_score_list = []
    for player in player_list:
        u = batted_played(player, date)
        v = cent_fifties(player, date)
        w = 0.3*v + 0.7*batting_avg(player, date)
        career_score = u*w
        M = ngiveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat', n=4)
        if len(M)==0:
            recent_score = 0
        else:
            recent_score = M['runs'].mean()
        career_score_list.append(career_score)
        recent_score_list.append(recent_score)
    
    career_score = scale_scores(career_score_list)
    recent_score = scale_scores(recent_score_list)
    batsman_score = 0.35*np.float64(career_score) + 0.65*np.float64(recent_score)
    return batsman_score

def bowled_played(player, date):
    x = match_data[match_data['match_dt']<date].sort_values(by='match_dt', ascending=False)[['match_id', 'team_roster_ids']]
    x['player_played'] = x['team_roster_ids'].apply(lambda x: 1 if player in x else 0)
    matches_played = x[x['player_played'] == 1]['match_id'].values

    x = bowler_data[bowler_data['match_dt']<date].sort_values(by='match_dt', ascending=False)[['match_id', 'bowler_id']]
    matches_bowled = x[x['bowler_id']==float(player)]['match_id'].values

    for i in matches_bowled:
        if i not in matches_played:
            matches_bowled = np.delete(matches_bowled, np.where(matches_bowled==i))
    
    return np.sqrt(len(matches_bowled)/(len(matches_played)+1))

def wickets_fwhaul(player, date):
    df_rel = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')
    df_rel['fwhaul'] = np.where(df_rel['wicket_count']>=5, 1, 0)
    return 10*np.sum(df_rel['fwhaul']) + np.sum(df_rel['wicket_count'])

def average_economy(player, date):
    df_rel = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')
    if len(df_rel)==0:
        return 115
    else:
        average = df_rel['runs'].sum()/(df_rel['wicket_count'].sum()+1)
        economy = df_rel['economy'].mean()
        return (average+1)*(economy+1)

def bowlerScore(player_list, date):
    bowler_score_list = []
    for player in (player_list):
        M = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')
        if len(M)>0:
            u = bowled_played(player, date)
            v = wickets_fwhaul(player, date)
            w = average_economy(player, date)
            bowler_score = u*v/w
            bowler_score_list.append(bowler_score)
        else:
            bowler_score_list.append(0)
    return bowler_score_list

def rel_strength(player_list1, player_list2, batsman_score_list, bowler_score_list):
    batsman_score = scale_scores(batsman_score_list)
    bowler_score = scale_scores(bowler_score_list)
    bat_strenght_team1 = np.sum(batsman_score[:len(player_list1)])
    bat_strenght_team2 = np.sum(batsman_score[len(player_list1):len(player_list1) + len(player_list2)])

    bowl_strenght_team1 = np.sum(bowler_score[:len(player_list1)])
    bowl_strenght_team2 = np.sum(bowler_score[len(player_list1):len(player_list1) + len(player_list2)])

    if bowl_strenght_team2 == 0 or bowl_strenght_team1 == 0:
        return 1
    else:
        return (bat_strenght_team1/bowl_strenght_team2) - (bat_strenght_team2/bowl_strenght_team1)

In [66]:
# Ratio invariant features
test_data['groundAvgWicketsLast15'] = test_data.apply(lambda x: avgWicketsGround(x['ground_id'], x['match_dt'], 15), axis=1)

test_data['lightAvgRunsLast15'] = test_data.apply(lambda x: lightAvgRunsLast15(x['lighting'], x['match_dt'], 15), axis=1)

test_data['lightAvgWicketsLast15'] = test_data.apply(lambda x: lightAvgWicketsLast15(x['lighting'], x['match_dt'], 15), axis=1)

test_data['seriesAvgRunsLast15'] = test_data.apply(lambda x: seriesAvgRunsLast15(x['series_name'], x['match_dt'], 15), axis=1)

test_data['seriesAvgWicketsLast15'] = test_data.apply(lambda x: seriesAvgWicketsLast15(x['series_name'], x['match_dt'], 15), axis=1)

test_data['inn1AvgRunsLast15'] = test_data.apply(lambda x: inn1AvgRunsLast15(x['match_dt'], 15), axis=1)

test_data['inn2AvgRunsLast15'] = test_data.apply(lambda x: inn2AvgRunsLast15(x['match_dt'], 15), axis=1)

test_data['inn1AvgWicketsLast15'] = test_data.apply(lambda x: inn1AvgWicketsLast15(x['match_dt'], 15), axis=1)

test_data['inn2AvgWicketsLast15'] = test_data.apply(lambda x: inn2AvgWicketsLast15(x['match_dt'], 15), axis=1)

In [67]:
# Match level features

# team1tossWinnerWins
test_data['team1tossWinnerWins'] = test_data.apply(lambda x: team1tossWinnerWins(x['team1_id'], x['team2_id'], x['match_dt']), axis=1)

# team1BatsFirstWins
test_data['team1BatsFirstWins'] = test_data.apply(lambda x: team1BatsFirstWins(x['team1_id'], x['team2_id'], x['match_dt']), axis=1)

# team1BatsSecondWins
test_data['team1BatsSecondWins'] = test_data.apply(lambda x: team1BatsSecondWins(x['team1_id'], x['team2_id'], x['match_dt']), axis=1)

# team1WinpLight
test_data['team1WinpLight'] = test_data.apply(lambda x: team1WinpLight(x['team1_id'], x['team2_id'], x['match_dt'], x['lighting']), axis=1)

# team1WinpSeries
test_data['team1WinpSeries'] = test_data.apply(lambda x: team1WinpSeries(x['team1_id'], x['team2_id'], x['match_dt'], x['series_name']), axis=1)

# team1AvgRunsMargin
test_data['team1AvgRunsMargin'] = test_data.apply(lambda x: team1AvgRunsMargin(x['team1_id'], x['team2_id'], x['match_dt'], 15), axis=1)

# team1AvgWicketsMargin
test_data['team1AvgWicketsMargin'] = test_data.apply(lambda x: team1AvgWicketsMargin(x['team1_id'], x['team2_id'], x['match_dt'], 15), axis=1)

# team1AvgWicketsLost
test_data['team1AvgWicketsLost'] = test_data.apply(lambda x: team1AvgWicketsLost(x['team1_id'], x['team2_id'], x['match_dt'],15), axis=1)

# team1AvgRR
test_data['team1AvgRR'] = test_data.apply(lambda x: team1AvgRR(x['team1_id'], x['team2_id'], x['match_dt'],15), axis=1)

In [68]:
# Player level features

#team1PlayerOfMatch
test_data['team1PlayerOfMatchLast15'] = test_data.apply(lambda x: team1PlayerOfMatchLast15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

#team1TotalRunsPOMLast15
test_data['team1TotalRunsPOMLast15'] = test_data.apply(lambda x: team1TotalRunsPOMLast15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

#team1TotalWicketsPOMLast15
test_data['team1TotalWicketsPOMLast15'] = test_data.apply(lambda x: team1TotalWicketsPOMLast15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

#team1SRover120Last15
test_data['team1SRover120Last15'] = test_data.apply(lambda x: team1SRover120Last15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

#team1CaptainRuns
test_data['team1CaptainRuns'] = test_data.apply(lambda x: team1CaptainRuns(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

#team1BattingAvgLast15
test_data['team1BattingAvgLast15'] = test_data.apply(lambda x: team1BattingAvgLast15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

#team1BoundaryLast15
test_data['team1BoundaryLast15'] = test_data.apply(lambda x: team1BoundaryLast15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

#team1BatBoundaryPercentLast15
test_data['team1BatBoundaryPercentLast15'] = test_data.apply(lambda x: team1BatBoundaryPercentLast15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

#team1EconBelow8Last15
test_data['team1EconBelow8Last15'] = test_data.apply(lambda x: team1EconBelow8Last15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

#team1DotPercentLast15
test_data['team1DotPercentLast15'] = test_data.apply(lambda x: team1DotPercentLast15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

#team1BowlBoundaryPercentLast15
test_data['team1BowlBoundaryPercentLast15'] = test_data.apply(lambda x: team1BowlBoundaryPercentLast15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

#team1MaidenPercent
test_data['team1MaidenPercent'] = test_data.apply(lambda x: team1MaidenPercent(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

#team1ExtrasPercentLast15
test_data['team1ExtrasPercentLast15'] = test_data.apply(lambda x: team1ExtrasPercentLast15(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'],15), axis=1)

In [69]:
# Team score features

test_data['bowlerScore'] = test_data.apply(lambda x: bowlerScore(x['team_roster_ids'], x['match_dt']), axis=1)

test_data['batsmanScore'] = test_data.apply(lambda x: batsmanScore(x['team_roster_ids'], x['match_dt']), axis=1)

test_data['rel_strength'] = test_data.apply(lambda x: rel_strength(x['team1_roster_ids'], x['team2_roster_ids'], x['batsmanScore'], x['bowlerScore']), axis=1)

test_data.drop(['bowlerScore', 'batsmanScore', 'team_roster_ids'], axis=1, inplace=True)

In [70]:
batsman_data['bowler_details'] = batsman_data[batsman_data['bowler_details'].isna() == False].apply(lambda x: x['bowler_details'].split(':')[1:3], axis=1)
batsman_data['batsman_details'] = batsman_data[batsman_data['batsman_details'].isna() == False].apply(lambda x: x['batsman_details'].split(':')[1:3], axis=1)
bowler_data['bowler_details'] = bowler_data[bowler_data['bowler_details'].isna() == False].apply(lambda x: x['bowler_details'].split(':')[1:3], axis=1)

In [71]:
def team1DeathBoundaries(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['over_faced_first', 'Fours', 'Sixes']]
        df_rel1 = df_rel1[df_rel1['over_faced_first'] > 16]
        df_rel1['boundaries'] = df_rel1['Fours'] + df_rel1['Sixes']
        res_list1.append(np.sum(df_rel1['boundaries']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['over_faced_first', 'Fours', 'Sixes']]
        df_rel2 = df_rel2[df_rel2['over_faced_first'] > 16]
        df_rel2['boundaries'] = df_rel2['Fours'] + df_rel2['Sixes']
        res_list2.append(np.sum(df_rel2['boundaries']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)
    
def team1DeathRuns(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['over_faced_first', 'runs']]
        df_rel1 = df_rel1[df_rel1['over_faced_first'] > 16]
        res_list1.append(np.sum(df_rel1['runs']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['over_faced_first', 'runs']]
        df_rel2 = df_rel2[df_rel2['over_faced_first'] > 16]
        res_list2.append(np.sum(df_rel2['runs']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)

def team1Dravidian(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['balls_faced', 'runs']]
        df_rel1 = df_rel1[df_rel1['balls_faced'] > 15]
        res_list1.append(np.sum(df_rel1['runs']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['balls_faced', 'runs']]
        df_rel2 = df_rel2[df_rel2['balls_faced'] > 15]
        res_list2.append(np.sum(df_rel2['runs']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)

def team1RunsLeft(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['batsman_details', 'runs']]
        if len(df_rel1)>0:
            flag = ['Left' in i[0] for i in df_rel1['batsman_details'].values]
            df_rel1 = df_rel1[flag]
            res_list1.append(np.sum(df_rel1['runs']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['batsman_details', 'runs']]
        if len(df_rel2)>0:
            flag = ['Left' in i[0] for i in df_rel2['batsman_details'].values]
            df_rel2 = df_rel2[flag]
            res_list2.append(np.sum(df_rel2['runs']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)
    
def team1RunsRight(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['batsman_details', 'runs']]
        if len(df_rel1)>0:
            flag = ['Right' in i[0] for i in df_rel1['batsman_details'].values]
            df_rel1 = df_rel1[flag]
            res_list1.append(np.sum(df_rel1['runs']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['batsman_details', 'runs']]
        if len(df_rel2)>0:
            flag = ['Right' in i[0] for i in df_rel2['batsman_details'].values]
            df_rel2 = df_rel2[flag]
            res_list2.append(np.sum(df_rel2['runs']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)
 

In [72]:
def team1WicketsLeft(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel1)>0:
            flag = ['Left' in i[1] for i in df_rel1['bowler_details'].values]
            df_rel1 = df_rel1[flag]
            res_list1.append(np.sum(df_rel1['wicket_count']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel2)>0:
            flag = ['Left' in i[1] for i in df_rel2['bowler_details'].values]
            df_rel2 = df_rel2[flag]
            res_list2.append(np.sum(df_rel2['wicket_count']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)
    
def team1WicketsRight(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel1)>0:
            flag = ['Right' in i[1] for i in df_rel1['bowler_details'].values]
            df_rel1 = df_rel1[flag]
            res_list1.append(np.sum(df_rel1['wicket_count']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel2)>0:
            flag = ['Right' in i[1] for i in df_rel2['bowler_details'].values]
            df_rel2 = df_rel2[flag]
            res_list2.append(np.sum(df_rel2['wicket_count']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)
    
def team1WicketsFast(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel1)>0:
            flag = ['fast' in i[1] for i in df_rel1['bowler_details'].values]
            df_rel1 = df_rel1[flag]
            res_list1.append(np.sum(df_rel1['wicket_count']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel2)>0:
            flag = ['fast' in i[1] for i in df_rel2['bowler_details'].values]
            df_rel2 = df_rel2[flag]
            res_list2.append(np.sum(df_rel2['wicket_count']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)
    
def team1WicketsOffbreak(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel1)>0:
            flag = ['offbreak' in i[1] for i in df_rel1['bowler_details'].values]
            df_rel1 = df_rel1[flag]
            res_list1.append(np.sum(df_rel1['wicket_count']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel2)>0:
            flag = ['offbreak' in i[1] for i in df_rel2['bowler_details'].values]
            df_rel2 = df_rel2[flag]
            res_list2.append(np.sum(df_rel2['wicket_count']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)
    
def team1WicketsOrthodox(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel1)>0:
            flag = ['orthodox' in i[1] for i in df_rel1['bowler_details'].values]
            df_rel1 = df_rel1[flag]
            res_list1.append(np.sum(df_rel1['wicket_count']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel2)>0:
            flag = ['orthodox' in i[1] for i in df_rel2['bowler_details'].values]
            df_rel2 = df_rel2[flag]
            res_list2.append(np.sum(df_rel2['wicket_count']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)
    
def team1WicketsGoogly(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel1)>0:
            flag = ['googly' in i[1] for i in df_rel1['bowler_details'].values]
            df_rel1 = df_rel1[flag]
            res_list1.append(np.sum(df_rel1['wicket_count']))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bowl')[['bowler_details', 'wicket_count']]
        if len(df_rel2)>0:
            flag = ['googly' in i[1] for i in df_rel2['bowler_details'].values]
            df_rel2 = df_rel2[flag]
            res_list2.append(np.sum(df_rel2['wicket_count']))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list1)/np.sum(res_list2)

def team1CaptainWickets(player_list1, player_list2, date):

    res_list1 = []
    for player in player_list1:
        df_rel1 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['is_bowler_captain']]
        df_rel1 = df_rel1[df_rel1['is_bowler_captain'] == 1]
        res_list1.append(len(df_rel1))

    res_list2 = []
    for player in player_list2:
        df_rel2 = giveLastNgamesPlayer(player_id=player, date=date, bat_or_bowl='bat')[['is_bowler_captain']]
        df_rel2 = df_rel2[df_rel2['is_bowler_captain'] == 1]
        res_list2.append(len(df_rel2))
            
    if np.sum(res_list2) == 0 or np.sum(res_list1) == 0:
        return 1
    else:
        return np.sum(res_list2)/np.sum(res_list1)

In [73]:
# test_data['team1DeathRuns'] = test_data.apply(lambda x: team1DeathRuns(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

# test_data['team1DeathBoundaries'] = test_data.apply(lambda x: team1DeathBoundaries(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

# test_data['team1Dravidian'] = test_data.apply(lambda x: team1Dravidian(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

test_data['team1RunsLeft'] = test_data.apply(lambda x: team1RunsLeft(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

test_data['team1RunsRight'] = test_data.apply(lambda x: team1RunsRight(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

In [74]:
test_data['team1WicketsLeft'] = test_data.apply(lambda x: team1WicketsLeft(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

test_data['team1WicketsRight'] = test_data.apply(lambda x: team1WicketsRight(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

test_data['team1WicketsFast'] = test_data.apply(lambda x: team1WicketsFast(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

test_data['team1WicketsOffbreak'] = test_data.apply(lambda x: team1WicketsOffbreak(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

test_data['team1WicketsOrthodox'] = test_data.apply(lambda x: team1WicketsOrthodox(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

test_data['team1WicketsGoogly'] = test_data.apply(lambda x: team1WicketsGoogly(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

test_data['team1CaptainWickets'] = test_data.apply(lambda x: team1CaptainWickets(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

In [83]:
cols = list(test_data.columns[13:])

In [84]:
X_test = test_data[cols]

In [86]:
with open('models//4//ensemble-cat.pkl', 'rb') as file:
    ensemble = pickle.load(file)

In [87]:
feature_desc = {
    'team1Num50Last15':'Ratio of team1 to team2 number of 50s by players in last 15 games.',
    'team1WinpLast5':'Ratio of team1 to team2 percent  win in last 5 games.',
    'team1OnlyAvgRunsLast15':'Average inning runs of team1 only in last 15 games.',
    'team1WinpLast15F2F':'Team1 win percent against Team2 in last 15 games.',
    'groundAvgRunsLast15':'Average runs scored in the ground in last 15 games.',
    'groundAvgWicketsLast15':'Average wickets lost in the ground in last 15 games.',
    'lightAvgRunsLast15':'Average runs scored in the lighting in last 15 games.',
    'lightAvgWicketsLast15':'Average wickets lost in the lighting in last 15 games.',
    'seriesAvgRunsLast15':'Average runs scored in the series in last 15 games.',
    'seriesAvgWicketsLast15':'Average wickets lost in the series in last 15 games.',
    'inn1AvgRunsLast15':'Average runs scored in inning 1 in last 15 games.',
    'inn2AvgRunsLast15':'Average runs scored in inning 2 in last 15 games.',
    'inn1AvgWicketsLast15':'Average wickets lost in inning 1 in last 15 games.',
    'inn2AvgWicketsLast15':'Average wickets lost in inning 2 in last 15 games.',
    'team1tossWinnerWins': 'Ratio of team1 to team2 percent of toss wins resulting in match wins.',
    'team1BatsFirstWins': 'Ratio of team1 to team2 percent of match wins resulting from batting first.',
    'team1BatsSecondWins': 'Ratio of team1 to team2 percent of match wins resulting from batting second.',
    'team1WinpLight': 'Ratio of team1 to team2 percent win given the lighting condition.',
    'team1WinpSeries': 'Ratio of team1 to team2 percent win given the series name',
    'team1AvgRunsMargin' : 'Ratio of team1 to team2 average of margin of runs won by',
    'team1AvgWicketsMargin' : 'Ratio of team1 to team2 average of margin of wickets won by',
    'team1AvgWicketsLost' :  'Ratio of team1 to team2 avg wickets lost w.r.t total balls faced.',
    'team1AvgRR' : 'Ratio of team1 to team2 average runs rate',
    'team1PlayerOfMatchLast15' : 'Ratio of team1 to team2 total number of player of the match won by players in last 15 games.',
    'team1TotalRunsPOMLast15' : 'Ratio of team1 to team2 total runs by player of the match in last 15 games.',
    'team1TotalWicketsPOMLast15' : 'Ratio of team1 to team2 total wickets by player of the match in last 15 games.',
    'team1SRover120Last15': 'Ratio of team1 to team2 percent player-level strike rate over 120 in last 15 games.',
    'team1CaptainRuns' : 'Ratio of team1 to team2 total runs scored by captains.',
    'team1BoundaryLast15' : 'Ratio of team1 to team2 total boundaries in last 15 games.',
    'team1BatBoundaryPercentLast15' : 'Ratio of team1 to team2 percent of runs scored that are boundaries in last 15 games.',
    'team1EconBelow8Last15' : 'Ratio of team1 to team2 percent player-level economy rate below 8 in last 15 games.',
    'team1DotPercentLast15' : 'Ratio of team1 to team2 percent of balls that are dots in last 15 games.',
    'team1BowlBoundaryPercentLast15' : 'Ratio of team1 to team2 percent of runs conceded that are boundaries in last 15 games.',
    'team1MaidenPercent': 'Ratio of team1 to team2 percent of maidens bowled.',
    'team1ExtrasPercentLast15' : 'Ratio of team1 to team2 percent of extras conceded in last 15 games.',
    'rel_strength' : 'Relative strength of team1 to team2.',
    'team1RunsLeft':'Ratio of team1 to team2 runs scored by left-handed batsmen.',
    'team1RunsRight':'Ratio of team1 to team2 runs scored by right-handed batsmen.',
    'team1WicketsLeft':'Ratio of team1 to team2 wickets taken by left-arm bowlers.',
    'team1WicketsRight':'Ratio of team1 to team2 wickets taken by right-arm bowlers.',
    'team1WicketsFast':'Ratio of team1 to team2 wickets taken by fast bowlers.',
    'team1WicketsOffbreak':'Ratio of team1 to team2 wickets taken by off-spinners.',
    'team1WicketsOrthodox':'Ratio of team1 to team2 wickets taken by orthodox spinners.',
    'team1WicketsGoogly':'Ratio of team1 to team2 wickets taken by googly bowlers.',
    'team1CaptainWickets':'Ratio of team1 to team2 wickets taken by captains.'
}

In [88]:
estimators = ensemble.named_estimators_

feature_importance = {}
for _, est in estimators.items():
    if hasattr(est, 'feature_importances_'):
        norm_imp = est.feature_importances_ / sum(est.feature_importances_)
        for i, imp in zip(cols,norm_imp):
            feature_importance.setdefault(i, []).append(imp)

feature_importance = {k: sum(v) / len(v) for k, v in feature_importance.items()}
feature_importance = list(zip(feature_importance, feature_importance.values()))
feature_importance.sort(key=lambda x: x[1], reverse=True)
feature_importance = pd.DataFrame(feature_importance, columns=['feat_name', 'model_feat_imp_train']).head(10)
feature_importance['feat_description'] = feature_importance['feat_name'].map(feature_desc)
feature_importance['feat_id'] = [i+1 for i in feature_importance.index]
feature_importance['feat_rank_train'] = [i+1 for i in feature_importance.index]

In [90]:
test_data['y_pred_01'] = ensemble.predict(X_test)

test_data['win_pred_score'] = ensemble.predict_proba(X_test)[:,1]

test_data['win_pred_score'] = np.where( (test_data['y_pred_01']==0), (1-test_data['win_pred_score']), test_data['win_pred_score'])

test_data['win_pred_team_id'] = np.where( (test_data['y_pred_01']==1), (test_data['team1_id']), test_data['team2_id'])

In [91]:
def train_hps_depth(ensemble):
    res = []
    for i in ensemble.estimators_:
        if hasattr(i, 'max_depth'):
            res.append(f"{i.max_depth}")
        elif i.get_param('depth') is not None:
            res.append(f"{i.get_param('depth')}")
    return ";".join(res)

def train_hps_trees(ensemble):
    res = []
    for i in ensemble.estimators_:
        if hasattr(i, 'n_estimators'):
            res.append(f"{i.n_estimators}")
        elif i.get_param('iterations') is not None:
            res.append(f"{i.get_param('iterations')}")
    return ";".join(res)

def train_hps_lr(ensemble):
    res = []
    for i in ensemble.estimators_:
        if hasattr(i, 'learning_rate'):
            res.append(f"{i.learning_rate}")
        elif i.get_param('learning_rate') is not None:
            res.append(f"{i.get_param('learning_rate')}")
    return ";".join(res)

def isEnsemble(ensemble):
    if len(ensemble.estimators_) == 1:
        return 'no'
    else:
        return 'yes'

In [92]:
def df_file1(ensemble):
    test_data['dataset_type'] = 'r2'
    test_data['train_algorithm'] = ';'.join(list(ensemble.named_estimators_.keys()))
    test_data['is_ensemble'] = isEnsemble(ensemble)
    test_data['train_hps_trees'] = train_hps_trees(ensemble)
    test_data['train_hps_depth'] = train_hps_depth(ensemble)
    test_data['train_hps_lr'] = train_hps_lr(ensemble)
    test_data['match id'] = test_data['match_id']

    df_file1 = pd.concat([test_data[['match id','dataset_type','win_pred_team_id','win_pred_score','train_algorithm', 'is_ensemble', 'train_hps_trees', 'train_hps_depth', 'train_hps_lr'] + list(feature_importance['feat_name'].head(10))]])

    renaming_dict = {}
    for i,col in enumerate(list(feature_importance['feat_name'].head(10))):
        renaming_dict[col] = f'indep_feat_id{i+1}'
    df_file1.rename(columns=renaming_dict, inplace=True)

    for i in range(1,11):
        if f'indep_feat_id{i}' not in df_file1.columns:
            df_file1[f'indep_feat_id{i}'] = np.nan
    return df_file1

In [93]:
df_file1 = df_file1(ensemble)

In [97]:
df_file2 = feature_importance[['feat_id', 'feat_name', 'feat_description', 'model_feat_imp_train','feat_rank_train']]

In [100]:
df_file1.to_csv('sub//r2//2024_DS_Track_File1_SATOSHI NAKAMOTO.csv', index=False)
df_file2.to_csv('sub//r2//2024_DS_Track_File2_SATOSHI NAKAMOTO.csv', index=False)